# 抽出型文書要約

## 手続き

1. 前準備
1. クリーニング(文区切り，絵文字/記号などの削除)
1. 文章の正規化(全角半角変換，数字の削除)
1. 文章の単語分割(形態素解析)
1. 単語の正規化(同じ意味を持つ単語の統一)
1. ストップワードの除去(名詞/形容詞/副詞/動詞のみの利用，辞書による除去)
1. 文書のベクトル表現(TF-IDF)
1. 要約モデルの適用(LexRank)
1. FlaskでAPIに

## 実装

### 1. 前準備

In [23]:
!pip3 install tinysegmenter

  Running setup.py bdist_wheel for tinysegmenter ... done
  Stored in directory: /root/.cache/pip/wheels/68/71/2b/6402196bf28012826e507ef7b99df6ebd98cce78bd99023471
Successfully built tinysegmenter


In [9]:
import numpy as np
import pandas as pd

import neologdn

# 文区切り
import functools
from ja_sentence_segmenter.common.pipeline import make_pipeline
from ja_sentence_segmenter.concatenate.simple_concatenator import concatenate_matching
from ja_sentence_segmenter.normalize.neologd_normalizer import normalize
from ja_sentence_segmenter.split.simple_splitter import split_newline, split_punctuation

# クリーニング
import re
from modules import data_cleaning as dc

# MeCab(形態素解析)
import MeCab as mecab
mecab = mecab.Tagger('-r /etc/mecabrc -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')

# 単語の正規化
import emoji
import mojimoji

# ストップワード除去
# 名詞/形容詞/副詞/動詞のみを抽出できるライブラリのインポート
import mecabpr
mpr = mecabpr.MeCabPosRegex('-r /etc/mecabrc -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')

# 抽出型要約モデル(LexRank)
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

### 2. クリーニング

In [44]:
split_punc2 = functools.partial(split_punctuation, punctuations=r"。!?")
concat_tail_te = functools.partial(concatenate_matching, former_matching_rule=r"^(?P<result>.+)([\r\n]+)$", remove_former_matched=False)
segmenter = make_pipeline(normalize, split_newline, concat_tail_te, split_punc2)

text_org = """
長岡は美味しいラーメン屋がたくさんある。\nその中でも特に美味しい(と個人的に思う)ラーメン屋を紹介したいと思う。\nそれは…「おこじょ」である。\nここは，限定ラーメンを月1ペースで出しており，何度行っても新しい 味が楽しめるラーメン屋である。\n現在(2021/9)は，うなぎハモラーメンを提供している。\nこれは，スープが泡立っているため，大変軽い舌触りであり，味もクリーミーに仕上がっている。\nトッピングのハモは，ナスと一緒に食 べることが推奨されており，ふわふわかつジューシーな味わいが楽しめる。\nお値段は1,500円と少し高めだが，値段分の価値は十分だと考えている。\n皆さんも機会があればぜひ食べてもらいたい。
"""

In [45]:
# 『楽しみ〜〜〜』の『〜〜〜』などを削除
text = neologdn.normalize(text_org)

In [46]:
sentences = list(segmenter(text))
pd.DataFrame(sentences)

,0
0,長岡は美味しいラーメン屋がたくさんある。
1,その中でも特に美味しい(と個人的に思う)ラーメン屋を紹介したいと思う。
2,それは…「おこじょ」である。
3,"ここは,限定ラーメンを月1ペースで出しており,何度行っても新しい味が楽しめるラーメン屋である。"
4,"現在(2021/9)は,うなぎハモラーメンを提供している。"
5,"これは,スープが泡立っているため,大変軽い舌触りであり,味もクリーミーに仕上がっている。"
6,"トッピングのハモは,ナスと一緒に食べることが推奨されており,ふわふわかつジューシーな味わいが..."
7,"お値段は1,500円と少し高めだが,値段分の価値は十分だと考えている。"
8,皆さんも機会があればぜひ食べてもらいたい。


### 3. 文章の正規化

In [47]:
sentences_dc = dc.data_cleaning(sentences)
pd.DataFrame(sentences_dc)

,0
0,長岡は美味しいラーメン屋がたくさんある。
1,その中でも特に美味しいと個人的に思うラーメン屋を紹介したいと思う。
2,それはおこじょである。
3,ここは限定ラーメンを月0ペースで出しており何度行っても新しい味が楽しめるラーメン屋である。
4,現在0はうなぎハモラーメンを提供している。
5,これはスープが泡立っているため大変軽い舌触りであり味もクリーミーに仕上がっている。
6,トッピングのハモはナスと一緒に食べることが推奨されておりふわふわかつジューシーな味わいが楽しめる。
7,お値段は0円と少し高めだが値段分の価値は十分だと考えている。
8,皆さんも機会があればぜひ食べてもらいたい。


### 4.5.6. 文章の単語分割，単語の正規化，ストップワード除去

In [48]:
sentence_words = []
result = []
for i, sentence in enumerate(sentences_dc):
    # 名詞|形容詞|副詞|動詞のみの抽出
    ma = sum(mpr.findall(sentence, "(名詞|形容詞|副詞|動詞)", raw=True), [])
    # 活用形の統一(基本形へ)
    sentence_ma = []
    for word in ma:
        if word.split(',')[6] == "*":
            if not bool(re.search(r'[a-zA-Z]',word.split('\t')[0])):
                sentence_ma.append(word.split('\t')[0])
            else :
                sentence_ma.append(re.sub(r"[a-zA-Z]", "", word.split('\t')[0]))
                    
        elif bool(re.search(r'[a-zA-Z]',word.split(',')[6])):
            if not bool(re.search(r'[a-zA-Z]',word.split(',')[7])):
                sentence_ma.append(word.split(',')[7])
            else :
                sentence_ma.append(re.sub(r"[a-zA-Z]", "", word.split(',')[7]))
        else:
            sentence_ma.append(word.split(',')[6])
    result.append(sentence_ma)

In [49]:
result

[['長岡', '美味しい', 'ラーメン屋', 'たくさん', 'ある'],
 ['中', '特に', '美味しい', '個人的', '思う', 'ラーメン屋', '紹介', 'する', '思う'],
 ['それ', 'おこじょ'],
 ['ここ',
  '限定',
  'ラーメン',
  '月',
  '0',
  'ペース',
  '出す',
  'おる',
  '何',
  '度',
  '行う',
  '新しい',
  '味',
  '楽しめる',
  'ラーメン屋'],
 ['現在', '0', 'うなぎ', 'ハモ', 'ラーメン', '提供', 'する', 'いる'],
 ['これ',
  'スープ',
  '泡立つ',
  'いる',
  'ため',
  '大変',
  '軽い',
  '舌触り',
  '味',
  'クリーミー',
  '仕上がる',
  'いる'],
 ['トッピング',
  'ハモ',
  'ナス',
  '一緒',
  '食べる',
  'こと',
  '推奨',
  'する',
  'れる',
  'おる',
  'ふわふわ',
  'ジューシー',
  '味わい',
  '楽しめる'],
 ['値段', '0円', '少し', '高め', '値段', '分', '価値', '十分', '考える', 'いる'],
 ['皆さん', '機会', 'ある', 'ぜひ', '食べる', 'もらう']]

In [50]:
corpus = [' '.join(sentence) + u'。' for sentence in result]
"".join(corpus)

'長岡 美味しい ラーメン屋 たくさん ある。中 特に 美味しい 個人的 思う ラーメン屋 紹介 する 思う。それ おこじょ。ここ 限定 ラーメン 月 0 ペース 出す おる 何 度 行う 新しい 味 楽しめる ラーメン屋。現在 0 うなぎ ハモ ラーメン 提供 する いる。これ スープ 泡立つ いる ため 大変 軽い 舌触り 味 クリーミー 仕上がる いる。トッピング ハモ ナス 一緒 食べる こと 推奨 する れる おる ふわふわ ジューシー 味わい 楽しめる。値段 0円 少し 高め 値段 分 価値 十分 考える いる。皆さん 機会 ある ぜひ 食べる もらう。'

In [51]:
corpus

['長岡 美味しい ラーメン屋 たくさん ある。',
 '中 特に 美味しい 個人的 思う ラーメン屋 紹介 する 思う。',
 'それ おこじょ。',
 'ここ 限定 ラーメン 月 0 ペース 出す おる 何 度 行う 新しい 味 楽しめる ラーメン屋。',
 '現在 0 うなぎ ハモ ラーメン 提供 する いる。',
 'これ スープ 泡立つ いる ため 大変 軽い 舌触り 味 クリーミー 仕上がる いる。',
 'トッピング ハモ ナス 一緒 食べる こと 推奨 する れる おる ふわふわ ジューシー 味わい 楽しめる。',
 '値段 0円 少し 高め 値段 分 価値 十分 考える いる。',
 '皆さん 機会 ある ぜひ 食べる もらう。']

### 7. 8.文書のベクトル表現，要約モデル(Lexrank)の適用

In [52]:
# 文章要約メソッド
def summy_test(sentences_org, corpus, sum_count):
    sentences = [' '.join(sentence) + u'。' for sentence in corpus]
    text_prep = "".join(sentences)
    parser = PlaintextParser.from_string(text_prep, Tokenizer('japanese'))

    summarizer = LexRankSummarizer()
    summarizer.stop_words = ['']
    # sentences_countにて、sentence（文章の数）を選びます。
    # 例えばsentences_countを３にすると文章を3文に要約してくれます。
    summary = summarizer(document=parser.document, sentences_count=sum_count)
    
    print(u'文書要約結果')    
    b = []
    for sentence in summary:
        b.append(sentences_org[sentences.index("{}".format(sentence.__str__()))])
        b.append("\n")
    return "".join(b)

In [53]:
print(summy_test(sentences, result, sum_count=3))

文書要約結果
長岡は美味しいラーメン屋がたくさんある。
それは…「おこじょ」である。
ここは,限定ラーメンを月1ペースで出しており,何度行っても新しい味が楽しめるラーメン屋である。



### 9. モジュールの動作確認

In [32]:
%load_ext autoreload
%autoreload 2
from modules import extractive_summarization as es

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
text = """
長岡は美味しいラーメン屋がたくさんある．
その中でも特に美味しい(と個人的に思う)ラーメン屋を紹介したいと思う．
それは…「おこじょ」である．
ここは，限定ラーメンを月1ペースで出しており，何度行っても新しい味が楽しめるラーメン屋である．
現在(2021/9)は，うなぎハモラーメンを提供している．
これは，スープが泡立っているため，大変軽い舌触りであり，味もクリーミーに仕上がっている．
トッピングのハモは，ナスと一緒に食べることが推奨されており，ふわふわかつジューシーな味わいが楽しめる．
お値段は1,500円と少し高めだが，値段分の価値は十分だと考えている．
皆さんも機会があればぜひ食べてもらいたい．
"""

In [36]:
summary = es.preprocessed_lexrank(text, sum_count=3)

In [37]:
print(summary)

長岡は美味しいラーメン屋がたくさんある。
それは…「おこじょ」である。
ここは,限定ラーメンを月1ペースで出しており,何度行っても新しい味が楽しめるラーメン屋である。

